# Homework 1

## Shackles of the Sea: The Pirate's Hold

Here's a map of the ship.

![image.png](https://res.cloudinary.com/dvidz8bwi/image/upload/v1706668686/CIS700/700hw1_tmer6i.png)

### Import Text Adventures

In [188]:
from text_adventure_games import (
    games, parsing, actions, things, blocks, viz
)

## Locations

In [189]:

hold = things.Location(
    "Cargo Hold",
    "You are in the cargo Hold. It's a dimly lit, cramped space, filled with the creaking sounds of the ship and the musty smell of seawater mixed with aged wood. Shadows cast by flickering lanterns dance over stacks of crates, barrels, and a haphazard assortment of goods seized by the pirates. The air is heavy with the scent of brine and the faint trace of old spices.\n"
)
deck = things.Location(
    "Main Deck",
    "You are on the main deck. The deck is a spacious, open area bathed in moonlight, offering a panoramic view of the endless sea. The salty sea breeze whispers through the sails, and the stars above provide a faint, serene light. The wooden floorboards groan underfoot, echoing the rhythmic waves. It's a gateway to all the ship's key areas.\n",
)
quarter = things.Location(
    "Pirates’ Quarters",
    "You are in a disorderly, cramped room filled with rows of rough-hewn bunks. The air is thick with the odor of rum, sweat, and the remnants of cheap tobacco. Scraps of old maps, empty bottles, and tattered clothes litter the floor. It's a chaotic jumble of pirate life, replete with secrets and potential disguises.\n"
)
cabin = things.Location(
    "Captain’s Cabin",
    "You are in a luxury room. Lavishly furnished with exotic carpets and dark, polished wood. The walls are adorned with maps and the spoils of piracy. A large desk, cluttered with papers and trinkets, dominates the room. The ever-present risk of encountering the captain adds a palpable tension to the air, a contrast to the cabin's luxury.\n",
)
guarded_entrance = things.Location(
    "Guarded Entrance",
    "You can see the main entrance of the ship, constantly guarded by stern-looking pirates. Torches on either side cast a flickering light, creating an intimidating barrier to freedom.\n"
)
exit = things.Location(
    "Locked Exit",
    "You are looking at a heavy door, an alternative exit unguarded, but secured with a sturdy lock.\n",
)
outside = things.Location(
    "Outside",
    "You are outside the ship. The sea is calm, and the moonlight casts a serene glow over the water.\n"
)
outside.set_property("winning", True)


# Map of Locations
hold.add_connection("up", deck)
deck.add_connection("east", quarter)
deck.add_connection("west", cabin)
deck.add_connection("north", guarded_entrance)
deck.add_connection("south", exit)
guarded_entrance.add_connection("out", outside)
exit.add_connection("out", outside)



## Items

### Gettable Items

In [190]:
nail = things.Item(
    "nail",
    "a rusty nail",
    "A SLIGHTLY BENT NAIL WITH A SHARP POINT AND A RUSTED SURFACE.",
)
# hold.add_item(nail)
nail.set_property("is_takeable", True)
# nail.set_property("is_sharp", True)

clothes = things.Item(
    "clothes",
    "pirate clothes",
    "ROUGH, WEATHERED GARMENTS TYPICAL OF A PIRATE, ESSENTIAL FOR BLENDING IN UNNOTICED.",
)
quarter.add_item(clothes)
clothes.set_property("is_takeable", True)
clothes.add_command_hint("put on")

key = things.Item(
    "key",
    "a brass key",
    "AN ORNATE, BRASS KEY WITH INTRICATE ENGRAVINGS.",
)
cabin.add_item(key)
key.set_property("is_takeable", True)


### Scenery Items

_We called these "non-gettable" in the tutorial_

In [191]:
# Places to search:
# Broken Furniture: A collection of damaged chairs, splintered tables, and other discarded furniture pieces. Amidst the debris, there might be useful items like the Rusty Nail.
# Boxes and Chests: A variety of wooden boxes and old chests.
# Discarded Personal Belongings: A heap of personal effects left behind by previous captives or careless pirates.

boxes = things.Item(
    "boxes",
    "wooden boxes",
    "A VARIETY OF WOODEN BOXES AND OLD CHESTS.",
)
boxes.set_property("is_takeable", False)
boxes.set_property("is_searchable", True)
boxes.set_property("contains_nail", False)
boxes.add_command_hint("search boxes")
hold.add_item(boxes)

furniture = things.Item(
    "furniture",
    "broken furniture",
    "A COLLECTION OF DAMAGED CHAIRS, SPLINTERED TABLES, AND OTHER DISCARDED FURNITURE PIECES.",
)
furniture.set_property("is_takeable", False)
furniture.set_property("is_searchable", True)
furniture.set_property("contains_nail", True)
furniture.add_command_hint("search furniture")
hold.add_item(furniture)

belongings = things.Item(
    "belongings",
    "discarded personal belongings",
    "A HEAP OF PERSONAL EFFECTS LEFT BEHIND BY PREVIOUS CAPTIVES OR CARELESS PIRATES.",
)
belongings.set_property("is_takeable", False)
belongings.set_property("is_searchable", True)
belongings.set_property("contains_nail", False)
belongings.add_command_hint("search belongings")
hold.add_item(belongings)

door = things.Item(
    "door",
    "a heavy door",
    "A HEAVY DOOR, AN ALTERNATIVE EXIT.",
)
door.set_property("is_takeable", False)
door.set_property("is_locked", True)
door.add_command_hint("unlock door")
door.add_command_hint("out")
exit.add_item(door)

entrance = things.Item(
    "entrance",
    "the main entrance",
    "THE MAIN ENTRANCE OF THE SHIP.",
)
entrance.set_property("is_takeable", False)
entrance.set_property("is_guarded", True)
entrance.add_command_hint("out")
guarded_entrance.add_item(entrance)

## Characters

### Player

In [192]:
# Player
player = things.Character(
    name="You",
    description="You can't remember your identity, but that's not the most urgent thing at the moment.",
    persona="I am on an escape.",
)
player.set_property("is_tied", True)
player.set_property("is_disguised", False)

### Non-Player Characters (NPCs)

## Actions

- Unlock_Door
- Search
- Untie
- Disguise

In [193]:
class Unlock_Door(actions.Action):
    ACTION_NAME = "unlock door"
    ACTION_DESCRIPTION = "Unlock a door with a key"
    ACTION_ALIASES = ['open door', 'use key']
    
    def __init__(self, game, command):
        super().__init__(game)
        # TODO - your code here
        self.character = player
        self.key = self.parser.match_item('key', self.parser.get_items_in_scope(self.character))
        self.door = self.parser.match_item('door', self.parser.get_items_in_scope(self.character))

    def check_preconditions(self) -> bool:
        """
        Preconditions:
        * There must be a door
        * The character must be at the same location as the door
        * The door must be locked
        * The character must have the key in their inventory
        """
        # TODO - your code here
        if not self.was_matched(self.key):
            description = "The key couldn't be found."
            self.parser.fail(description)
            return False
        if not self.was_matched(self.door):
            description = "The door couldn't be found."
            self.parser.fail(description)
            return False
        if not self.character.location.here(self.door):
            description = "The character must be at the same location as the door."
            self.parser.fail(description)
            return False
        if not self.door.get_property("is_locked"):
            description = "The door is not locked."
            self.parser.fail(description)
            return False
        if not self.character.is_in_inventory(self.key):
            description = "You don't have the key."
            self.parser.fail(description)
            return False
        return True

    def apply_effects(self):
        """
        Effects:
        * Unlocks the door
        """
        # TODO - your code here
        self.door.set_property("is_locked", False)
        self.parser.ok("You opened the door with the key.\n")

In [194]:
class Search(actions.Action):
    ACTION_NAME = "search"
    ACTION_DESCRIPTION = "Search the place"
    ACTION_ALIASES = []

    def __init__(self, game, command):
        super().__init__(game)
        self.character = player
        self.place = self.parser.match_item(command, self.parser.get_items_in_scope(self.character))
        

    def check_preconditions(self) -> bool:
        if not self.was_matched(self.place):
            description = "There's no such place to search."
            self.parser.fail(description)
            return False
        if not self.place.get_property("is_searchable"):
            description = "You can't search this place."
            self.parser.fail(description)
            return False
        return True

    def apply_effects(self):
        if self.place.get_property("contains_nail"):
            self.parser.ok("You searched the {place}.\nYou found a nail.\n".format(place=self.place.name))
            self.place.set_property("contains_nail", False)
            player.add_to_inventory(nail)
        else:
            self.parser.ok("You searched the {place}.\nYou found nothing.\n".format(place=self.place.name))


In [195]:
class Untie(actions.Action):
    ACTION_NAME = "untie"
    ACTION_DESCRIPTION = "Untie yourself"
    ACTION_ALIASES = ['cut rope', 'free']

    def __init__(self, game, command):
        super().__init__(game)
        self.character = player
        self.nail = nail
        

    def check_preconditions(self) -> bool:
        if not self.was_matched(self.nail):
            description = "You tried to untie yourself but failed. You need a sharp thing to do this.\n"
            self.parser.fail(description)
            return False
        if not self.character.is_in_inventory(self.nail):
            description = "You tried to untie yourself but failed. You need a sharp thing to do this.\n"
            self.parser.fail(description)
            return False
        return True

    def apply_effects(self):
        self.character.set_property("is_tied", False)
        self.parser.ok("You untied yourself with the nail. You are free now.\n")


In [196]:
class Disguise(actions.Action):
    ACTION_NAME = "put on"
    ACTION_DESCRIPTION = "Disguise yourself"
    ACTION_ALIASES = ['wear', 'put on', 'disguise']

    def __init__(self, game, command):
        super().__init__(game)
        self.character = player
        self.clothes = self.parser.match_item('clothes', self.parser.get_items_in_scope(self.character))
        

    def check_preconditions(self) -> bool:
        if not self.was_matched(self.clothes):
            description = "There's nothing to disguise yourself with.\n"
            self.parser.fail(description)
            return False
        return True

    def apply_effects(self):
        self.character.add_to_inventory(clothes)
        self.character.set_property("is_disguised", True)
        self.parser.ok("You disguised yourself with the clothes you found. You look just like a pirate now.\n")


## Blocks


In [197]:
class Tie_Block(blocks.Block):

    def __init__(self, location: things.Location):
        super().__init__(
            "Your bindings prevent you from taking the stairs", "You tried to go upstairs, but fell down. Your bindings prevent you from taking the stairs.\nMaybe try untie yourself first.\n"
        )
        self.location = location

    def is_blocked(self) -> bool:
        if player.get_property("is_tied"):
            return True
        return False


In [198]:
class Guard_Block(blocks.Block):

    def __init__(self, location: things.Location):
        super().__init__(
            "A guard is around the entrance", "The guard sees you and yells for help. They found you!\nYou are captured by the pirates again.\n\nYOU LOST!"
        )
        self.location = location

    def is_blocked(self) -> bool:
        if player.get_property("is_disguised"):
            return False
        player.set_property("is_dead", True)
        return True

In [199]:
class Door_Block(blocks.Block):

    def __init__(self, location: things.Location, door: things.Item):
        super().__init__("A locked door blocks your way", "The door ahead is locked.\n")
        self.location = location
        self.door = door

    def is_blocked(self) -> bool:
        if self.door.get_property("is_locked"):
            return True
        return False

In [200]:
import random

class Captain_Block(blocks.Block):

    def __init__(self, location: things.Location):
        super().__init__("You don't know if the captain is in his cabin...", "The captain is in their cabin. They spot you!\nYou are captured by the pirates again.\n\nYOU LOST!")
        self.location = location

    def is_blocked(self) -> bool:
        captain_is_in = random.choice([True, False])
        if captain_is_in:
            player.set_property("is_dead", True)
            return True
        return False

In [201]:
hold.add_block("up", Tie_Block(hold))
guarded_entrance.add_block("out", Guard_Block(guarded_entrance))
exit.add_block("out", Door_Block(exit, door))
deck.add_block("west", Captain_Block(cabin))

## Play Action Castle

In [202]:
class ActionCastle(games.Game):
    def __init__(
        self, start_at: things.Location, player: things.Character, characters=None,
        custom_actions=None
    ):
        super().__init__(start_at, player, characters=characters, custom_actions=custom_actions)

    def is_won(self) -> bool:
        if self.player.location.get_property("winning"):
            self.parser.ok("You are free!")
            return True
        return False

In [203]:

characters = []
custom_actions = [Unlock_Door, Search, Untie, Disguise]

# The Game
game = ActionCastle(hold, player, characters=characters, custom_actions=custom_actions)


In [204]:
print("You are kidnapped by a group of pirates. Escape the ship!\n")

game.game_loop()

You are kidnapped by a group of pirates. Escape the ship!

You are in the cargo Hold. It's a dimly lit, cramped space, filled with the
creaking sounds of the ship and the musty smell of seawater mixed with aged
wood. Shadows cast by flickering lanterns dance over stacks of crates, barrels,
and a haphazard assortment of goods seized by the pirates. The air is heavy with
the scent of brine and the faint trace of old spices.

Exits:
Up to Main Deck

You see:
 * wooden boxes
        search boxes
 * broken furniture
        search furniture
 * discarded personal belongings
        search belongings


You searched the furniture.
You found a nail.

You untied yourself with the nail. You are free now.

You are on the main deck. The deck is a spacious, open area bathed in moonlight,
offering a panoramic view of the endless sea. The salty sea breeze whispers
through the sails, and the stars above provide a faint, serene light. The wooden
floorboards groan underfoot, echoing the rhythmic waves. I